In [7]:
import pandas as pd
import json
import minsearch

# Ingestion

In [8]:
with open('../data/arsonor_chunks_id.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [9]:
documents[15]

{'article_id': '3632a3b4',
 'title': 'L’intelligence artificielle (IA) dans le studio de production audio (5/6)',
 'category': 'LA POST-PROD',
 'tags': 'dé-mixage, de-noise, de-reverb, deep learning, plug-in audio, restauration audio, stems',
 'chunk_id': '3632a3b4-11',
 'chunk_text': 'A sa sortie en 2016, Regroover reçut plusieurs prix d’innovation et fut considéré comme le futur du sampling. Enfin, le bundle ERA , sorti en 2018, est la gamme de produits d’Accusonus qui connaît la croissance la plus rapide. Avec des plugins, comme le « Noise Remover » ou le « Voice leveler » , toujours plus nombreux et rassemblés en une seule interface, le « Audio Clean-up Assistant » , l’ERA Bundle s’impose peu à peu comme un sérieux concurrent à iZotope RX dans le domaine des logiciels de nettoyage et restauration sonore. La simplicité d’utilisation des plugins (souvent munis d’un simple potard à tourner), permet aux créateurs débutants et professionnels d’améliorer instantanément leurs enregistreme

In [10]:
index = minsearch.Index(
    text_fields=['title', 'tags', 'chunk_text'],
    keyword_fields=['article_id', 'category', 'chunk_id']
)

In [11]:
index.fit(documents)

# RAG flow

In [5]:
from openai import OpenAI

client = OpenAI()

In [2]:
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'

In [6]:
response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": query}]
    )
    
response.choices[0].message.content

"Créer une musique dans un home-studio nécessite un certain équipement pour garantir une qualité audio satisfaisante. Voici une liste des matériels essentiels pour commencer :\n\n### 1. **Ordinateur**\n   - Un ordinateur puissant (PC ou Mac) capable de faire tourner un logiciel de production audio (DAW).\n\n### 2. **Logiciel de Production Audio (DAW)**\n   - Choisissez un logiciel tel que Ableton Live, FL Studio, Logic Pro, Cubase, ou Pro Tools.\n\n### 3. **Interface Audio**\n   - Permet de connecter des instruments et des microphones à votre ordinateur tout en assurant une meilleure qualité sonore.\n\n### 4. **Moniteurs de Studio**\n   - Des enceintes de monitoring pour écouter votre musique de manière précise (par exemple, KRK, Yamaha, ou JBL).\n\n### 5. **Casque Audio**\n   - Un bon casque de studio pour un suivi audio précis (par exemple, Sennheiser, Audio-Technica).\n\n### 6. **Microphone**\n   - Un micro de bonne qualité pour enregistrer des voix ou des instruments (par exemple, 

In [20]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=30
    )

    return results

In [21]:
prompt_template = """
You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally add in your response the top 3 articles of arsonor (refer to the 'title') that are the best to read for answering this question.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
article title: {title}
article keywords: {tags}
content: {chunk_text}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [23]:
print(prompt)

You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally add in your response the top 3 articles of arsonor (refer to the 'title') that are the best to read for answering this question.

QUESTION: De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?

CONTEXT:
article title: Comment bien débuter en MAO: le home-studio démystifié
article keywords: DAW, hardware, home-studio, MAO, matériel audio, plugin, software
content: trouveras toujours quelqu’un pour te montrer ses 50 synthétiseurs analogiques, ses 10 boîtes à rythmes, ses 3 ordinateurs et quelques interfaces audio multi-pistes avec leurs micros, le tout pour plus de 10000€. La vérité :

In [17]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [24]:
question = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'
answer = rag(question)
print(answer)

Pour créer de la musique dans votre home-studio, vous n'avez pas besoin d'un équipement coûteux ou complexe. Voici les éléments essentiels nécessaires pour démarrer :

1. **Un ordinateur** : Cela peut être celui que vous avez déjà. Assurez-vous simplement qu'il soit relativement récent et capable de faire tourner des logiciels musicaux de manière fluide. Pensez à le configurer spécifiquement pour la musique, sans autres applications gourmandes en ressources comme des jeux.

2. **Un logiciel de production (DAW)** : C'est l'outil principal pour créer, enregistrer et manipuler votre musique. Il existe plusieurs DAW populaires comme Ableton Live, FL Studio, ou Reaper, que vous pouvez essayer.

3. **Un casque audio de bonne qualité** : Bien qu'un système de monitoring (enceintes) soit préférable pour un mixage précis, un bon casque suffit pour commencer à vous familiariser avec votre DAW et la production musicale.

4. **Vos oreilles** : C'est l'outil le plus important pour apprécier et juge

# Retrieval evaluation

# RAG evaluation